# Welcome to Our Graduation Project: **"Automatic Story Generation - Hakawaty"**

This notebook provides a detailed walkthrough of the steps involved in fine-tuning Llama 3.1 to achieve optimal performance in generating high-quality stories. After extensive experimentation with various parameters, we present our approach to create the best possible stories.

 This notebook was originally created by Unsloth, but modifications and additional elements have been added to better align with our project purpose and objectives.

# Fine-tuning Llama-3.1
    version: Llama-3.1--8B-instruct-4bit

**install unsloth**

unsloth is a library that makes  fine-tuning LLMs  x2 faster, use 70% less memory, and with no degradation in accuracy!

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:

from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # our dataset pompt-story has a maximum sequence length of 995 (closest to 1024) , that is why we choose 1024
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage=.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",

    "unsloth/Llama-3.2-1B-bnb-4bit",
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", # the model that we choose
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

**adding LoRA adapters**

so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0.3,
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.3.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.12.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


# Merge data
merge the original data (996) with the data that we generates (124)

⛔ DO NOT RUN

In [ ]:
import pandas as pd

file1 = pd.read_excel("/content/MSA_data.xlsx")
file2 = pd.read_excel("/content/125_generated_stories.xlsx")

merged_data = pd.concat([file1, file2], ignore_index=True)

# Save the merged data to a new Excel file
merged_data.to_excel("merged_file.xlsx", index=False)

print("Files have been merged successfully!")



Files have been merged successfully!


# data preparation


In [ ]:
!pip install pandas datasets


In [ ]:
import pandas as pd

file_path = "merged_file.xlsx"
data = pd.read_excel(file_path)

# prepare data to be split amd select the required columns
data = data[['Prompt', 'Story']]

1- prepare dataset in a **converasation format** that is compatible with HuggingFace and Llama (LLaMA expects a conversation-based input format).





In [ ]:
from datasets import Dataset

# Prepare data in HuggingFace format ("role", "content")
formatted_data = [
    {
        "conversations": [
            {"role": "user", "content": row['Prompt']},
            {"role": "assistant", "content": row['Story']}
        ]
    }
    for _, row in data.iterrows()
]


2- **split the formatted data** into training, validation, and test sets.
**Split dataset into:**
*   Train: 80%
*   Validation: 10%
*   Test: 10%

3- **convert to hugging face objects** which ensures compatibility with tools provided by HuggingFace, such as tokenizers and trainers.


4-  **Setting the chat template for Llama** (it expect a conversation), where it tells the tokenizer how to encode the data so the model interprets it properly,**then apply it to the data splits**.



> Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```



In [ ]:
from unsloth.chat_templates import get_chat_template
from sklearn.model_selection import train_test_split

# split data into train, val, and test
train_data, temp_data = train_test_split(
    formatted_data, test_size=0.2, random_state=42
)

val_data, test_data = train_test_split(
    temp_data, test_size=0.5, random_state=42
)

# Convert to HuggingFace Datasets objects
train_dataset = Dataset.from_list(train_data)  # Train dataset
val_dataset = Dataset.from_list(val_data)  # Validation dataset
test_dataset = Dataset.from_list(test_data)  # Test dataset



# Set the chat template to LLaMA-3.1
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1"
)

# Apply the template to format the dataset
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Apply formatting
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)  # Train
val_dataset = val_dataset.map(formatting_prompts_func, batched=True)  # Validation
test_dataset = test_dataset.map(formatting_prompts_func, batched=True)  # Test

# print to confirm sizes:   80% of 1120 , 10% of 1120 , 10% of 1120 -> should be 896, 112, 112
print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))
print("Test dataset size:", len(test_dataset))

Map:   0%|          | 0/896 [00:00<?, ? examples/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Train dataset size: 896
Validation dataset size: 112
Test dataset size: 112


In [ ]:
# Check samples
print(train_dataset[0]['conversations'])  # Check a sample conversation from train dataset
print(train_dataset[0]['text'])           # Check formatted text from train dataset
print(val_dataset[0]['conversations'])
print(val_dataset[0]['text'])
print(test_dataset[0]['conversations'])
print(test_dataset[0]['text'])

[{'content': 'أكتب قصة قصيرة تكون مفهومة للأطفال في عمر 3-5. نهاية القصة يجب أن تكون حزينة. يجب أن يكون هناك حوار. عدد الشخصيات في القصة يجب أن يكون 3. بلد الأحداث هي  ليبيا. النشاط لعب ألعاب الطاولة. يجب أن تحتوي القصة على شعور بالذنب.  أكتب القصة مباشرة.', 'role': 'user'}, {'content': 'كان هناك في بلد جميل يُدعى ليبيا، ثلاثة أصدقاء صغار يُدعون سامي، ريم، وعمر. كانوا يجتمعون كل يوم بعد الدراسة في بيت عمر الدافئ ليلعبوا ألعاب الطاولة الممتعة.\n\nذات يوم، اختاروا لعب لعبة جديدة اشتراها عمر مؤخرًا. كانت لعبة القطار التي تتطلب دورة وتفكيرًا لبناء مسارات القطارات. بدأوا بحماس وكانت الضحكات تملأ الغرفة.\n\n"هيا بنا نجعل القطار يمر عبر الجسر العالي!" قالت ريم بحماس.\n\nلكن سامي كان يشعر بالغيرة قليلًا لأنه لم يكن جيدًا في اللعبة مثل ريم. وفي لحظة من الإحباط، ضغط بيده على الطاولة مما أدى لانهيار الجسر وتحطم مسار القطار.\n\nريم بدت حزينة جدًا، وقالت: "كنا قريبين من الانتهاء، لكنك خربتها يا سامي!"\n\nشعر سامي بالذنب الشديد، لم يقصد أبدًا أن يجعل ريم حزينة. "أنا حقا آسف، لم أقصد أن أفسد اللعبة."

# Train the model
### using Huggingface TRL's `SFTTrainer` (Supervised Fine-tuning Trainer)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from transformers import EarlyStoppingCallback


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text", # The specific field in the dataset that contains the input text (as we formatted)
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer), # ensures all sequences in a batch are the same length.
    dataset_num_proc = 2, # number of processes used for dataset preprocessing. (parallel)
    packing = False, # mostly used in short sequence (which is not in our case)

    args = TrainingArguments(
        per_device_train_batch_size = 2, # batch size
        gradient_accumulation_steps = 4, # Number of steps to accumulate gradients before performing a backward pass.
        warmup_steps = 5, # Number of initial training steps (with a lower learning rate to stabilize training)
        num_train_epochs = 4, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10, # which training metrics are logged (such as loss).
        evaluation_strategy="steps",
        save_steps=50,  # Save model checkpoint every 50 steps
        eval_steps=10,
        optim = "adamw_8bit",
        weight_decay = 0.2, # Regularization technique to prevent overfitting (by penalizing large weights)
        lr_scheduler_type = "linear", # to adjust the learning rate during training.
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb",
        max_grad_norm = 2.0 ,# Maximum value for gradient clipping to prevent exploding gradients (help with overfitting)
        load_best_model_at_end=True,

    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=45)]  # Stop after 45 steps with no improvement
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/896 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/112 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_responses_only` method to only train on the assistant outputs and ignore the loss on the user's inputs.

"after testing the model (pilot expeirment), it was clear that the model understood the input and do kind of a good job, but the output was not as what we wish for. focusing on the output is the best in our case"

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/896 [00:00<?, ? examples/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"]) # input ID has the full sequence

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nأكتب قصة قصيرة تكون مفهومة للأطفال في عمر 3-5. أحداث القصة يجب أن تدور في الجزيرة. نهاية القصة يجب أن تكون حزينة. يجب أن يكون هناك حوار. عدد الشخصيات في القصة يجب أن يكون 1. الدرس المستفاد من القصة هو نحن نقرر كيفية الرد على المواقف الصعبة. بلد الأحداث هي  فلسطين. الأحداث تدور في فصل الخريف. أضف تحولا غير متوقع في الأحداث: الكشف عن أن الحدث الرئيسي كان خدعة. أكتب القصة مباشرة.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nفي جزيرة صغيرة وسط البحر، عاشت فتاة صغيرة تدعى ليلى. كانت ليلى تعيش وحيدة في بيت صغير، حوله الأشجار الملونة بأوراق الخريف الذهبية والحمراء. كانت تحب الجزيرة كثيرًا وتقضي وقتها في استكشاف كل ركن فيها.\n\nفي يوم خريفي هادئ، وجدت ليلى زجاجة صغيرة على الشاطئ، داخلها رسالة! فتحت الزجاجة بحماس، وبدأت تقرأ الرسالة: "عزيزتي ليلى، نحن نحتاج مساعدتك. جزيرتك في خطر وتحتاج إلى حامي. تعا

In [ ]:
tokenizer.decode(trainer.eval_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nأكتب قصة قصيرة تكون مفهومة للأطفال في عمر 8-10. أحداث القصة يجب أن تدور في المتحف. نهاية القصة يجب أن تكون سعيدة. يجب أن يكون هناك حوار. عدد الشخصيات في القصة يجب أن يكون 3. موضوع القصة يجب أن يكون الفن والإبداع. بلد الأحداث هي  قطر. يجب أن تحتوي القصة على شعور بالخوف.  أكتب القصة مباشرة.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nفي يوم صيفي جميل، قررت مها وأخوها سعيد وصديقهم يوسف زيارة المتحف الوطني في بلدهم قطر، لقد كانوا متحمسين لاستكشاف الفن والإبداع.\n\nوصلوا إلى المتحف وأعينهم تتلألأ بفضل الإثارة. بدأوا بجولة في أقسام المتحف المختلفة، وكانوا مبهورين باللوحات والتماثيل والقطع الأثرية التي تحكي قصصًا من الماضي.\n\nوفي لحظة، بينما كانوا يستكشفون قسم الفن المعاصر، لاحظوا لوحة كبيرة بألوان زاهية كانت معلقة بمفردها في أحد الزوايا. اللوحة كانت مختلفة عن الأخرى، فكانت تبدو وكأنها بوابة إلى 

Decodes the labels (target outputs) while replacing all masked tokens values with a space token (space).


`x == -100` : This checks whether a token has been masked.



In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                           \n\nفي جزيرة صغيرة وسط البحر، عاشت فتاة صغيرة تدعى ليلى. كانت ليلى تعيش وحيدة في بيت صغير، حوله الأشجار الملونة بأوراق الخريف الذهبية والحمراء. كانت تحب الجزيرة كثيرًا وتقضي وقتها في استكشاف كل ركن فيها.\n\nفي يوم خريفي هادئ، وجدت ليلى زجاجة صغيرة على الشاطئ، داخلها رسالة! فتحت الزجاجة بحماس، وبدأت تقرأ الرسالة: "عزيزتي ليلى، نحن نحتاج مساعدتك. جزيرتك في خطر وتحتاج إلى حامي. تعالي إلى الجبل الأعلى في الجزيرة، لنكشف لك السر." اهتز قلب ليلى بالقلق والحماس في نفس الوقت.\n\nقررت ليلى الذهاب إلى الجبل الأعلى، فأخذت معها حقيبة صغيرة مملوءة بالأمل والشجاعة. طوال الطريق، كانت تحدث نفسها: "أنا قوية، أستطيع حماية جزيرتي." الرحلة كانت طويلة ومتعبة، لكن ليلى لم تستسلم.\n\nوصلت أخيرًا إلى قمة الجبل، لكنها وجدت مفاجأة غريبة؛ لم يكن هناك أحد في انتظارها. بدأت تشعر بخيبة أمل وحزن، لقد كانت الرسالة خدعة. جلست ليلى على صخرة كبيرة، تنظ

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.eval_dataset[5]["labels"]])

'                                                                                                                                        \n\nفي يوم صيفي جميل، قررت مها وأخوها سعيد وصديقهم يوسف زيارة المتحف الوطني في بلدهم قطر، لقد كانوا متحمسين لاستكشاف الفن والإبداع.\n\nوصلوا إلى المتحف وأعينهم تتلألأ بفضل الإثارة. بدأوا بجولة في أقسام المتحف المختلفة، وكانوا مبهورين باللوحات والتماثيل والقطع الأثرية التي تحكي قصصًا من الماضي.\n\nوفي لحظة، بينما كانوا يستكشفون قسم الفن المعاصر، لاحظوا لوحة كبيرة بألوان زاهية كانت معلقة بمفردها في أحد الزوايا. اللوحة كانت مختلفة عن الأخرى، فكانت تبدو وكأنها بوابة إلى عالم آخر.\n\nمها، بفضولها عصرت كتف يوسف قائلة: "ألا تظنون أن هذه اللوحة فريدة؟" \n\nنعم، لكنها تبدو غريبة بعض الشيء"، أضاف سعيد بنبرة تحمل شيئًا من الخوف.\n\nفجأة، أحسوا بنسمة هواء باردة تسربت من حول اللوحة. ولوهلة، شعروا وكأن اللوحة تدعوهم إليها. يوسف، الذي كان دائم التحدي، أقدم خطوة نحو اللوحة، ومد يده ليمس الألوان الزاهية. وعندما فعل ذلك، امتدت الألوان نحو يده كالماء، مما جعل الأطفال يت

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
6.004 GB of memory reserved.


# Start Training

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 896 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 448
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
10,2.272700,1.521059
20,1.467100,1.521070
30,1.401600,1.504048
40,1.345200,1.487339
50,1.334000,1.483649
60,1.299000,1.484106
70,1.301500,1.475520
80,1.311400,1.472900
90,1.320700,1.466078
100,1.310800,1.464709


KeyboardInterrupt: 

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'start_gpu_memory' is not defined

# Inference



In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference (from unsloth)

prompt = """أكتب قصة قصيرة تكون مفهومة للأطفال في عمر 5-8.
أحداث القصة يجب أن تدور في منزل ضخم.
نهاية القصة يجب أن تكون سعيدة.
يجب أن يكون هناك حوار.
عدد الشخصيات في القصة يجب أن يكون 2.
بلد الأحداث هي المملكة العربية السعودية.
الأحداث تدور في فصل الصيف.
يجب أن تحتوي القصة على درس مفيد.
أكتب القصة مباشرة باللغة العربية."""

# Prepare the input message to he Llama format given the prompt
messages = [
    {"role": "user", "content": prompt},
]

# Apply the Llama chat template and tokenize
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True, # Adds something like [Assistant: ] at the end, to let or help the model to start its response.
    return_tensors="pt", # pytorch tensor (so the tokenizer ensures the inputs are directly compatible with PyTorch-based models)
).to("cuda")


# attention mask (to ignore padding tokens)
attention_mask = (inputs != tokenizer.pad_token_id).long()

# Generate the response from the model (the first try for temp and min_p values, we then expeirments with different values until we got the best which was : temp = 0.5 , min_p = 0.3)
outputs = model.generate(
    input_ids=inputs,
    attention_mask=attention_mask,
    max_new_tokens=1024,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

# Decode the model's output
generated_story = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print the result
print(generated_story[0])


system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

أكتب قصة قصيرة تكون مفهومة للأطفال في عمر 5-8.
أحداث القصة يجب أن تدور في منزل ضخم.
نهاية القصة يجب أن تكون سعيدة.
يجب أن يكون هناك حوار.
عدد الشخصيات في القصة يجب أن يكون 2.
بلد الأحداث هي المملكة العربية السعودية.
الأحداث تدور في فصل الصيف.
يجب أن تحتوي القصة على درس مفيد.
أكتب القصة مباشرة باللغة العربية.assistant

في يومٍ صيفي حار بالمملكة العربية السعودية، كان هناك منزل ضخم يقع في قلب مدينة صحراويةٍ جميلة، حيث النخيل يركض خلف الحارس، والشمس تحلق مثل صديقةً مخفية بين العيون والأحجار. وفي هذا المنزل العظيم، عاش والد وابنه، الفارس الكبير وريان.

كانت الأيام مليئة بالمرح والاستكشاف داخل الحديقة الكبيرة التي تؤمثُ جبال الصحراء. وفي يومٍ ما، قال ريان وهو يتحدث مع والده عن حلمه بالبحث عن كنوز مدينة النخيل:

"يا 아버지، هل تذكرين عندما كانت الحكايات تعرض أمامنا في المساء حول الكنوز المخفیة في المدينة؟ أحلم بأن أجد كنزًا مثل ذلك يومًا."

واجهته النظرة الواسعة والمرحة من والده: "عزيزي، يا ريان. الكنوز ليست الذهب والأ

# Saving, loading our fine-tuned model
use Huggingface's `push_to_hub` for an online save

In [ ]:
model.push_to_hub("Lamiia/hakawaty", token = "add_your_hf_token")
tokenizer.push_to_hub("Lamiia/hakawaty", token = "add_your_hf_token")

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Lamiia/hakawaty


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Loading

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Lamiia/hakawaty",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

prompt = """أكتب قصة قصيرة تكون مفهومة للأطفال في عمر 8-10. أحداث القصة يجب أن تدور في المحيط. نهاية القصة يجب أن تكون حزينة. يجب أن يكون هناك حوار. عدد الشخصيات في القصة يجب أن يكون 1. موضوع القصة يجب أن يكون الأساطير والخرافات. بلد الأحداث هي  لبنان. النشاط الرسم. أضف تحولا غير متوقع في الأحداث: الكشف عن أن الحدث الرئيسي كان خدعة. أكتب القصة مباشرة"""

messages = [
    {"role": "user", "content": prompt},
]


inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

attention_mask = (inputs != tokenizer.pad_token_id).long()

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024,
                   use_cache = True, temperature = 0.5, min_p = 0.1, attention_mask = attention_mask)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
في بلاد لبنان الجميلة، حيث البحر الأزرق يلمع تحت شمس الصباح الذهبية، كان يعيش سمكة صغيرة تدعى "لينا". لينا كانت مختلفة عن السمك الأخرى، فهي كانت تحب الرسم أكثر من السباحة في المياه الزرقاء. كل يوم بعد العودة من مغامرة جديدة، كانت لينا ترسم ما رأت، وكانت هذه الرسومات تحكي قصصًا عن أساطير وأحلامها.

ذات يوم، وجدت لينا رسالة غامضة على الشاطئ. كانت الرسالة تقول: "توجد كنزًا في أعماق البحر، لكنه يمكن أن يُفقد أبدًا إذا لم يُرسل رسالة إلى السماء". لينا، بفضولها الشديد، قررت البحث عن هذا الكنز، وبدأت رحلتها تحت الماء.

بينما كانت تنزل،